In [429]:
import pandas as pd
import numpy as np 
import sklearn.datasets as dta
import scipy.stats as st
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.decomposition import LatentDirichletAllocation
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models, Model

In [430]:
path = "../CO_Accidents.csv"
df = pd.read_csv(path)
df.head()

,Unnamed: 0,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,512415,A-512446,Source2,3,2022-09-08 13:03:18,2022-09-08 13:32:26,39.848122,-104.985306,NaN,NaN,...,False,False,False,False,False,False,Day,Day,Day,Day
1,512525,A-512565,Source2,1,2022-09-08 08:44:08,2022-09-08 09:13:40,39.776600,-105.062630,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day
2,512588,A-512637,Source2,2,2022-09-08 06:57:38,2022-09-08 07:27:23,39.899731,-104.868507,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day
3,512593,A-512643,Source2,1,2022-09-08 08:45:51,2022-09-08 09:15:28,39.618809,-104.773201,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day
4,512594,A-512644,Source2,3,2022-09-08 08:42:23,2022-09-08 09:11:57,39.725094,-105.012817,NaN,NaN,...,False,False,False,False,False,False,Day,Day,Day,Day


In [431]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90885 entries, 0 to 90884
Data columns (total 47 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             90885 non-null  int64  
 1   ID                     90885 non-null  object 
 2   Source                 90885 non-null  object 
 3   Severity               90885 non-null  int64  
 4   Start_Time             90885 non-null  object 
 5   End_Time               90885 non-null  object 
 6   Start_Lat              90885 non-null  float64
 7   Start_Lng              90885 non-null  float64
 8   End_Lat                47671 non-null  float64
 9   End_Lng                47671 non-null  float64
 10  Distance(mi)           90885 non-null  float64
 11  Description            90885 non-null  object 
 12  Street                 90727 non-null  object 
 13  City                   90877 non-null  object 
 14  County                 90885 non-null  object 
 15  St

In [432]:
df.describe()

,Unnamed: 0,Severity,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in)
count,9.088500e+04,90885.000000,90885.000000,90885.000000,47671.000000,47671.000000,90885.000000,89602.000000,73624.000000,89518.000000,89805.000000,89255.000000,84678.000000,62826.000000
mean,3.909138e+06,2.443902,39.537114,-105.088422,39.496544,-105.190225,0.908223,47.536737,41.517384,52.383889,25.790389,11.280650,8.511995,0.001544
std,2.172853e+06,0.658282,0.572171,0.727363,0.664135,0.911719,2.930441,22.323277,26.108202,26.653627,2.962336,9.507933,6.273117,0.014303
min,5.124150e+05,1.000000,36.993870,-109.047048,36.993840,-109.283380,0.000000,-89.000000,-89.000000,1.000000,16.820000,0.000000,0.000000,0.000000
25%,2.057112e+06,2.000000,39.473172,-105.053238,39.322652,-105.181332,0.000000,30.000000,21.000000,29.000000,24.170000,10.000000,5.000000,0.000000
50%,3.614846e+06,2.000000,39.710964,-104.980141,39.696670,-104.987440,0.037000,46.900000,38.000000,50.000000,24.500000,10.000000,7.000000,0.000000
75%,5.449918e+06,3.000000,39.785191,-104.847168,39.802100,-104.860438,0.849000,64.900000,63.000000,77.000000,29.830000,10.000000,11.500000,0.000000
max,7.728193e+06,4.000000,41.002288,-102.044781,41.056590,-102.030371,254.399994,104.000000,104.000000,100.000000,30.940000,90.000000,254.300000,1.050000


In [433]:
len(df)

90885

In [434]:
df_drop = df.drop(columns = ["Source", "ID", "Start_Lat", "Start_Lng", "Description", "Timezone", "Weather_Timestamp", "Airport_Code", "Country", "Zipcode", "Street", "City", "County", "State", "Distance(mi)", "End_Lat", "End_Lng"])
df_drop

,Unnamed: 0,Severity,Start_Time,End_Time,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,512415,3,2022-09-08 13:03:18,2022-09-08 13:32:26,93.0,93.0,11.0,24.37,10.0,NE,...,False,False,False,False,False,False,Day,Day,Day,Day
1,512525,1,2022-09-08 08:44:08,2022-09-08 09:13:40,86.0,86.0,13.0,24.43,10.0,WNW,...,False,False,False,False,True,False,Day,Day,Day,Day
2,512588,2,2022-09-08 06:57:38,2022-09-08 07:27:23,71.0,71.0,26.0,24.62,10.0,SW,...,False,False,False,False,True,False,Day,Day,Day,Day
3,512593,1,2022-09-08 08:45:51,2022-09-08 09:15:28,78.0,78.0,21.0,24.23,10.0,SW,...,False,False,False,False,True,False,Day,Day,Day,Day
4,512594,3,2022-09-08 08:42:23,2022-09-08 09:11:57,86.0,86.0,13.0,24.43,10.0,WNW,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90880,7728172,2,2019-08-23 16:23:28,2019-08-23 16:52:30,86.0,86.0,14.0,24.38,10.0,E,...,False,False,False,False,False,False,Day,Day,Day,Day
90881,7728181,3,2019-08-23 17:43:51,2019-08-23 18:13:31,80.0,80.0,27.0,24.97,10.0,S,...,False,False,False,False,False,False,Day,Day,Day,Day
90882,7728187,3,2019-08-23 18:52:03,2019-08-23 19:21:34,82.0,82.0,20.0,24.37,10.0,ESE,...,False,False,False,False,False,False,Day,Day,Day,Day
90883,7728192,4,2019-08-23 20:15:17,2019-08-23 20:43:52,76.0,76.0,36.0,25.19,10.0,NNE,...,False,False,False,False,False,False,Night,Night,Day,Day


In [435]:
df_drop = df_drop.drop_duplicates()
df_drop = df_drop.dropna(how='any')

In [436]:
# df_drop['ID'] = df_drop['ID'].apply(lambda x: x.replace('A-', ''))
# df_drop.head()


In [437]:
df_drop['Start_Time'] = df_drop['Start_Time'].apply(lambda x: x.replace('-', '').replace(':', ''))
df_drop.head()

,Unnamed: 0,Severity,Start_Time,End_Time,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,512415,3,20220908 130318,2022-09-08 13:32:26,93.0,93.0,11.0,24.37,10.0,NE,...,False,False,False,False,False,False,Day,Day,Day,Day
1,512525,1,20220908 084408,2022-09-08 09:13:40,86.0,86.0,13.0,24.43,10.0,WNW,...,False,False,False,False,True,False,Day,Day,Day,Day
2,512588,2,20220908 065738,2022-09-08 07:27:23,71.0,71.0,26.0,24.62,10.0,SW,...,False,False,False,False,True,False,Day,Day,Day,Day
3,512593,1,20220908 084551,2022-09-08 09:15:28,78.0,78.0,21.0,24.23,10.0,SW,...,False,False,False,False,True,False,Day,Day,Day,Day
4,512594,3,20220908 084223,2022-09-08 09:11:57,86.0,86.0,13.0,24.43,10.0,WNW,...,False,False,False,False,False,False,Day,Day,Day,Day


In [438]:
df_drop['Start_Time'].dtypes

dtype('O')

In [439]:
df_drop['Start_Time'] = pd.to_numeric(df_drop['Start_Time'], errors='coerce')
df_drop['Start_Time'].dtypes


dtype('float64')

In [440]:
df_drop['End_Time'] = df_drop['End_Time'].apply(lambda x: x.replace('-', '').replace(':', ''))
df_drop.head()

,Unnamed: 0,Severity,Start_Time,End_Time,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,512415,3,NaN,20220908 133226,93.0,93.0,11.0,24.37,10.0,NE,...,False,False,False,False,False,False,Day,Day,Day,Day
1,512525,1,NaN,20220908 091340,86.0,86.0,13.0,24.43,10.0,WNW,...,False,False,False,False,True,False,Day,Day,Day,Day
2,512588,2,NaN,20220908 072723,71.0,71.0,26.0,24.62,10.0,SW,...,False,False,False,False,True,False,Day,Day,Day,Day
3,512593,1,NaN,20220908 091528,78.0,78.0,21.0,24.23,10.0,SW,...,False,False,False,False,True,False,Day,Day,Day,Day
4,512594,3,NaN,20220908 091157,86.0,86.0,13.0,24.43,10.0,WNW,...,False,False,False,False,False,False,Day,Day,Day,Day


In [441]:
df_drop['End_Time'] = pd.to_numeric(df_drop['End_Time'], errors='coerce')
df_drop['End_Time'].dtypes

dtype('float64')

In [442]:
#df_drop['ID'] = df_drop['ID'].astype(float) 
# df_drop['ID'] = df_drop['ID'].astype(int)
# df_drop['ID'].dtypes

In [443]:
# df_drop[['Start_Time', 'End_Time']] = df_drop[['Start_Time', 'End_Time']].astype(str)
# df_drop['Start_Time'] = df_drop['Start_Time'].str.strip()
# df_drop['End_Time'] = df_drop['Start_Time'].str.strip()
# #df_drop['Start_Time'] = pd.to_datetime(df_drop['Start_Time'], format='%Y%m%d', errors='coerce')
# df_drop['Start_Time'] = pd.to_numeric(df_drop['Start_Time'], errors='coerce')
# df_drop['End_Time'] = pd.to_numeric(df_drop['End_Time'], errors='coerce')
# df_drop[['Start_Time', 'End_Time']] = df_drop[['Start_Time', 'End_Time']].astype(float)
 

In [444]:
df_drop[['Start_Time', 'End_Time']] = df_drop[['Start_Time', 'End_Time']].astype(str)
df_drop['Start_Time'] = df_drop['Start_Time'].str.strip()
df_drop['End_Time'] = df_drop['End_Time'].str.strip()
df_drop['Start_Time'] = pd.to_datetime(df_drop['Start_Time'], errors='coerce')
df_drop['End_Time'] = pd.to_datetime(df_drop['End_Time'], errors='coerce')
df_drop[['Start_Time', 'End_Time']] = df_drop[['Start_Time', 'End_Time']].apply(pd.to_numeric, errors='coerce')
df_drop[['Start_Time', 'End_Time']].dtypes

Start_Time    int64
End_Time      int64
dtype: object

In [445]:
non_numeric = df_drop['Temperature(F)'].apply(lambda x: isinstance(x, str))
non_numeric.value_counts()
df_drop.fillna(0)
df_drop['Temperature(F)'].info()

<class 'pandas.core.series.Series'>
Index: 61402 entries, 0 to 90884
Series name: Temperature(F)
Non-Null Count  Dtype  
--------------  -----  
61402 non-null  float64
dtypes: float64(1)
memory usage: 959.4 KB


In [446]:
df_drop['Wind_Direction'].value_counts()

Wind_Direction
CALM        7918
N           6202
S           5289
W           4076
NNW         3934
NNE         3291
SSE         3197
E           2996
SSW         2735
NW          2707
NE          2548
SE          2490
WSW         2445
SW          2437
WNW         2404
ESE         2280
ENE         2089
VAR         2026
North        222
East          45
South         41
West          20
Variable      10
Name: count, dtype: int64

In [447]:
df_drop['Wind_Direction'] = df_drop['Wind_Direction'].astype(str)
df_drop['Wind_Direction'] = df_drop['Wind_Direction'].str.upper()
df_drop['Wind_Direction'].value_counts()

Wind_Direction
CALM        7918
N           6202
S           5289
W           4076
NNW         3934
NNE         3291
SSE         3197
E           2996
SSW         2735
NW          2707
NE          2548
SE          2490
WSW         2445
SW          2437
WNW         2404
ESE         2280
ENE         2089
VAR         2026
NORTH        222
EAST          45
SOUTH         41
WEST          20
VARIABLE      10
Name: count, dtype: int64

In [448]:
#Collect non-numerics

non_num_df = df_drop.select_dtypes(include=['object', 'bool'])
print(non_num_df)


      Wind_Direction Weather_Condition  Amenity   Bump  Crossing  Give_Way  \
0                 NE     Partly Cloudy    False  False     False     False   
1                WNW              Fair    False  False     False     False   
2                 SW              Fair    False  False      True     False   
3                 SW              Fair    False  False      True     False   
4                WNW              Fair    False  False     False     False   
...              ...               ...      ...    ...       ...       ...   
90880              E     Mostly Cloudy    False  False     False     False   
90881              S            Cloudy    False  False     False     False   
90882            ESE            Cloudy    False  False     False     False   
90883            NNE     Partly Cloudy    False  False     False     False   
90884            NNE     Partly Cloudy    False  False     False     False   

       Junction  No_Exit  Railway  Roundabout  Station   Stop  

In [449]:
# df_drop['Temperature(F)'] = int(round(df_drop['Temperature(F)'], 0))
# df_drop.head()


In [450]:
df_drop['Temperature(F)'].fillna(0, inplace=True)
df_drop['Temperature(F)'] = np.ceil(df_drop['Temperature(F)'])
df_drop['Temperature(F)'] = df_drop['Temperature(F)'].apply(int)
#df_drop['Temperature(F)'] = round(df_drop['Temperature(F)'].apply(int))

df_drop

# df_drop['Temperature(F)'] = pd.to_numeric(df_drop['Temperature(F)'], errors='coerce')
# df_drop.dropna(subset=['Temperature(F)'], inplace=True)
# df_drop['Temperature(F)'] = df_drop['Temperature(F)'].astype(int)


,Unnamed: 0,Severity,Start_Time,End_Time,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,512415,3,-9223372036854775808,-9223372036854775808,93,93.0,11.0,24.37,10.0,NE,...,False,False,False,False,False,False,Day,Day,Day,Day
1,512525,1,-9223372036854775808,-9223372036854775808,86,86.0,13.0,24.43,10.0,WNW,...,False,False,False,False,True,False,Day,Day,Day,Day
2,512588,2,-9223372036854775808,-9223372036854775808,71,71.0,26.0,24.62,10.0,SW,...,False,False,False,False,True,False,Day,Day,Day,Day
3,512593,1,-9223372036854775808,-9223372036854775808,78,78.0,21.0,24.23,10.0,SW,...,False,False,False,False,True,False,Day,Day,Day,Day
4,512594,3,-9223372036854775808,-9223372036854775808,86,86.0,13.0,24.43,10.0,WNW,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90880,7728172,2,-9223372036854775808,-9223372036854775808,86,86.0,14.0,24.38,10.0,E,...,False,False,False,False,False,False,Day,Day,Day,Day
90881,7728181,3,-9223372036854775808,-9223372036854775808,80,80.0,27.0,24.97,10.0,S,...,False,False,False,False,False,False,Day,Day,Day,Day
90882,7728187,3,-9223372036854775808,-9223372036854775808,82,82.0,20.0,24.37,10.0,ESE,...,False,False,False,False,False,False,Day,Day,Day,Day
90883,7728192,4,-9223372036854775808,-9223372036854775808,76,76.0,36.0,25.19,10.0,NNE,...,False,False,False,False,False,False,Night,Night,Day,Day


In [451]:
df_drop['Wind_Chill(F)'].fillna(0, inplace=True)
df_drop['Wind_Chill(F)'] = np.ceil(df_drop['Wind_Chill(F)'])
df_drop['Wind_Chill(F)'] = df_drop['Wind_Chill(F)'].apply(int)
#df_drop['Wind_Chill(F)'] = round(df_drop['Wind_Chill(F)'].apply(int))
df_drop

,Unnamed: 0,Severity,Start_Time,End_Time,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,512415,3,-9223372036854775808,-9223372036854775808,93,93,11.0,24.37,10.0,NE,...,False,False,False,False,False,False,Day,Day,Day,Day
1,512525,1,-9223372036854775808,-9223372036854775808,86,86,13.0,24.43,10.0,WNW,...,False,False,False,False,True,False,Day,Day,Day,Day
2,512588,2,-9223372036854775808,-9223372036854775808,71,71,26.0,24.62,10.0,SW,...,False,False,False,False,True,False,Day,Day,Day,Day
3,512593,1,-9223372036854775808,-9223372036854775808,78,78,21.0,24.23,10.0,SW,...,False,False,False,False,True,False,Day,Day,Day,Day
4,512594,3,-9223372036854775808,-9223372036854775808,86,86,13.0,24.43,10.0,WNW,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90880,7728172,2,-9223372036854775808,-9223372036854775808,86,86,14.0,24.38,10.0,E,...,False,False,False,False,False,False,Day,Day,Day,Day
90881,7728181,3,-9223372036854775808,-9223372036854775808,80,80,27.0,24.97,10.0,S,...,False,False,False,False,False,False,Day,Day,Day,Day
90882,7728187,3,-9223372036854775808,-9223372036854775808,82,82,20.0,24.37,10.0,ESE,...,False,False,False,False,False,False,Day,Day,Day,Day
90883,7728192,4,-9223372036854775808,-9223372036854775808,76,76,36.0,25.19,10.0,NNE,...,False,False,False,False,False,False,Night,Night,Day,Day


In [452]:
df_drop['Humidity(%)'].fillna(0, inplace=True)
df_drop['Humidity(%)'] = np.ceil(df_drop['Humidity(%)'])
df_drop['Humidity(%)'] = df_drop['Humidity(%)'].apply(int)
#df_drop['Humidity(%)'] = round(df_drop['Humidity(%)'].apply(int))
df_drop

,Unnamed: 0,Severity,Start_Time,End_Time,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,512415,3,-9223372036854775808,-9223372036854775808,93,93,11,24.37,10.0,NE,...,False,False,False,False,False,False,Day,Day,Day,Day
1,512525,1,-9223372036854775808,-9223372036854775808,86,86,13,24.43,10.0,WNW,...,False,False,False,False,True,False,Day,Day,Day,Day
2,512588,2,-9223372036854775808,-9223372036854775808,71,71,26,24.62,10.0,SW,...,False,False,False,False,True,False,Day,Day,Day,Day
3,512593,1,-9223372036854775808,-9223372036854775808,78,78,21,24.23,10.0,SW,...,False,False,False,False,True,False,Day,Day,Day,Day
4,512594,3,-9223372036854775808,-9223372036854775808,86,86,13,24.43,10.0,WNW,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90880,7728172,2,-9223372036854775808,-9223372036854775808,86,86,14,24.38,10.0,E,...,False,False,False,False,False,False,Day,Day,Day,Day
90881,7728181,3,-9223372036854775808,-9223372036854775808,80,80,27,24.97,10.0,S,...,False,False,False,False,False,False,Day,Day,Day,Day
90882,7728187,3,-9223372036854775808,-9223372036854775808,82,82,20,24.37,10.0,ESE,...,False,False,False,False,False,False,Day,Day,Day,Day
90883,7728192,4,-9223372036854775808,-9223372036854775808,76,76,36,25.19,10.0,NNE,...,False,False,False,False,False,False,Night,Night,Day,Day


In [453]:
df_drop['Pressure(in)'].fillna(0, inplace=True)
df_drop['Pressure(in)'] = np.ceil(df_drop['Pressure(in)'])
df_drop['Pressure(in)'] = df_drop['Pressure(in)'].apply(int)
#df_drop['Pressure(in)'] = round(df_drop['Pressure(in)'].apply(int))
df_drop

,Unnamed: 0,Severity,Start_Time,End_Time,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,512415,3,-9223372036854775808,-9223372036854775808,93,93,11,25,10.0,NE,...,False,False,False,False,False,False,Day,Day,Day,Day
1,512525,1,-9223372036854775808,-9223372036854775808,86,86,13,25,10.0,WNW,...,False,False,False,False,True,False,Day,Day,Day,Day
2,512588,2,-9223372036854775808,-9223372036854775808,71,71,26,25,10.0,SW,...,False,False,False,False,True,False,Day,Day,Day,Day
3,512593,1,-9223372036854775808,-9223372036854775808,78,78,21,25,10.0,SW,...,False,False,False,False,True,False,Day,Day,Day,Day
4,512594,3,-9223372036854775808,-9223372036854775808,86,86,13,25,10.0,WNW,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90880,7728172,2,-9223372036854775808,-9223372036854775808,86,86,14,25,10.0,E,...,False,False,False,False,False,False,Day,Day,Day,Day
90881,7728181,3,-9223372036854775808,-9223372036854775808,80,80,27,25,10.0,S,...,False,False,False,False,False,False,Day,Day,Day,Day
90882,7728187,3,-9223372036854775808,-9223372036854775808,82,82,20,25,10.0,ESE,...,False,False,False,False,False,False,Day,Day,Day,Day
90883,7728192,4,-9223372036854775808,-9223372036854775808,76,76,36,26,10.0,NNE,...,False,False,False,False,False,False,Night,Night,Day,Day


In [454]:
df_drop['Visibility(mi)'].fillna(0, inplace=True)
df_drop['Visibility(mi)'] = np.ceil(df_drop['Visibility(mi)'])
df_drop['Visibility(mi)'] = df_drop['Visibility(mi)'].apply(int)
#df_drop['Visibility(mi)'] = round(df_drop['Visibility(mi)'].apply(int))
df_drop

,Unnamed: 0,Severity,Start_Time,End_Time,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,512415,3,-9223372036854775808,-9223372036854775808,93,93,11,25,10,NE,...,False,False,False,False,False,False,Day,Day,Day,Day
1,512525,1,-9223372036854775808,-9223372036854775808,86,86,13,25,10,WNW,...,False,False,False,False,True,False,Day,Day,Day,Day
2,512588,2,-9223372036854775808,-9223372036854775808,71,71,26,25,10,SW,...,False,False,False,False,True,False,Day,Day,Day,Day
3,512593,1,-9223372036854775808,-9223372036854775808,78,78,21,25,10,SW,...,False,False,False,False,True,False,Day,Day,Day,Day
4,512594,3,-9223372036854775808,-9223372036854775808,86,86,13,25,10,WNW,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90880,7728172,2,-9223372036854775808,-9223372036854775808,86,86,14,25,10,E,...,False,False,False,False,False,False,Day,Day,Day,Day
90881,7728181,3,-9223372036854775808,-9223372036854775808,80,80,27,25,10,S,...,False,False,False,False,False,False,Day,Day,Day,Day
90882,7728187,3,-9223372036854775808,-9223372036854775808,82,82,20,25,10,ESE,...,False,False,False,False,False,False,Day,Day,Day,Day
90883,7728192,4,-9223372036854775808,-9223372036854775808,76,76,36,26,10,NNE,...,False,False,False,False,False,False,Night,Night,Day,Day


In [455]:
df_drop['Wind_Speed(mph)'].fillna(0, inplace=True)
df_drop['Wind_Speed(mph)'] = np.ceil(df_drop['Wind_Speed(mph)'])
df_drop['Wind_Speed(mph)'] = df_drop['Wind_Speed(mph)'].apply(int)
#df_drop['Wind_Speed(mph)'] = round(df_drop['Wind_Speed(mph)'].apply(int))
df_drop


,Unnamed: 0,Severity,Start_Time,End_Time,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,512415,3,-9223372036854775808,-9223372036854775808,93,93,11,25,10,NE,...,False,False,False,False,False,False,Day,Day,Day,Day
1,512525,1,-9223372036854775808,-9223372036854775808,86,86,13,25,10,WNW,...,False,False,False,False,True,False,Day,Day,Day,Day
2,512588,2,-9223372036854775808,-9223372036854775808,71,71,26,25,10,SW,...,False,False,False,False,True,False,Day,Day,Day,Day
3,512593,1,-9223372036854775808,-9223372036854775808,78,78,21,25,10,SW,...,False,False,False,False,True,False,Day,Day,Day,Day
4,512594,3,-9223372036854775808,-9223372036854775808,86,86,13,25,10,WNW,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90880,7728172,2,-9223372036854775808,-9223372036854775808,86,86,14,25,10,E,...,False,False,False,False,False,False,Day,Day,Day,Day
90881,7728181,3,-9223372036854775808,-9223372036854775808,80,80,27,25,10,S,...,False,False,False,False,False,False,Day,Day,Day,Day
90882,7728187,3,-9223372036854775808,-9223372036854775808,82,82,20,25,10,ESE,...,False,False,False,False,False,False,Day,Day,Day,Day
90883,7728192,4,-9223372036854775808,-9223372036854775808,76,76,36,26,10,NNE,...,False,False,False,False,False,False,Night,Night,Day,Day


In [456]:
df_drop['Precipitation(in)'].fillna(0, inplace=True)
df_drop['Precipitation(in)'] = np.ceil(df_drop['Precipitation(in)'])
df_drop['Precipitation(in)'] = df_drop['Precipitation(in)'].apply(int)
df_drop['Precipitation(in)'] = round(df_drop['Precipitation(in)'].apply(int))
df_drop

,Unnamed: 0,Severity,Start_Time,End_Time,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,512415,3,-9223372036854775808,-9223372036854775808,93,93,11,25,10,NE,...,False,False,False,False,False,False,Day,Day,Day,Day
1,512525,1,-9223372036854775808,-9223372036854775808,86,86,13,25,10,WNW,...,False,False,False,False,True,False,Day,Day,Day,Day
2,512588,2,-9223372036854775808,-9223372036854775808,71,71,26,25,10,SW,...,False,False,False,False,True,False,Day,Day,Day,Day
3,512593,1,-9223372036854775808,-9223372036854775808,78,78,21,25,10,SW,...,False,False,False,False,True,False,Day,Day,Day,Day
4,512594,3,-9223372036854775808,-9223372036854775808,86,86,13,25,10,WNW,...,False,False,False,False,False,False,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90880,7728172,2,-9223372036854775808,-9223372036854775808,86,86,14,25,10,E,...,False,False,False,False,False,False,Day,Day,Day,Day
90881,7728181,3,-9223372036854775808,-9223372036854775808,80,80,27,25,10,S,...,False,False,False,False,False,False,Day,Day,Day,Day
90882,7728187,3,-9223372036854775808,-9223372036854775808,82,82,20,25,10,ESE,...,False,False,False,False,False,False,Day,Day,Day,Day
90883,7728192,4,-9223372036854775808,-9223372036854775808,76,76,36,26,10,NNE,...,False,False,False,False,False,False,Night,Night,Day,Day


In [457]:
df_drop['Wind_Direction'] = df_drop['Wind_Direction'].replace({'NNW': 'NW', 
                                                                'SSE': 'SE', 
                                                                'Calm':'CALM',
                                                                'SSW': 'SW',
                                                                'NNE': 'NE',
                                                                'WSW': 'SW',
                                                                'WNW': 'NW',
                                                                'ESE': 'SE',
                                                                'ENE': 'NE',
                                                                'S': 'SOUTH',
                                                                'N': 'NORTH',
                                                                'W': 'WEST',
                                                                'E': 'EAST'
                                                    })

# df_drop = df_drop[df_drop['Wind_Direction'] != 'VAR']
# df_drop = df_drop[df_drop['Wind_Direction'] != 'Variable']

df_drop['Wind_Direction'] = df_drop['Wind_Direction'].str.upper()

df_drop = df_drop[(df_drop['Wind_Direction'] != 'VAR') & (df_drop['Wind_Direction'] != 'Variable')]
df_drop['Wind_Direction'].value_counts()

Wind_Direction
NW          9045
SE          7967
NE          7928
CALM        7918
SW          7617
NORTH       6424
SOUTH       5330
WEST        4096
EAST        3041
VARIABLE      10
Name: count, dtype: int64

In [458]:
# Preprocess "Wind_Direction" column (one-hot encoding)
wind_encoder = OneHotEncoder(sparse_output=False)
wind_encoded = wind_encoder.fit_transform(df_drop[['Wind_Direction']])
wind_columns = wind_encoder.get_feature_names_out(['Wind_Direction'])
df_wind_encoded = pd.DataFrame(wind_encoded, columns=wind_columns)
df_wind_encoded.head()

,Wind_Direction_CALM,Wind_Direction_EAST,Wind_Direction_NE,Wind_Direction_NORTH,Wind_Direction_NW,Wind_Direction_SE,Wind_Direction_SOUTH,Wind_Direction_SW,Wind_Direction_VARIABLE,Wind_Direction_WEST
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [459]:
pd.options.display.max_rows=100
df_drop['Weather_Condition'].value_counts()

Weather_Condition
Fair                            23317
Mostly Cloudy                    9950
Partly Cloudy                    8396
Light Snow                       5561
Cloudy                           5353
Fair / Windy                     1017
Snow                              801
Light Rain                        760
Mostly Cloudy / Windy             551
Partly Cloudy / Windy             471
Fog                               440
Showers in the Vicinity           205
Heavy Snow                        204
Cloudy / Windy                    200
Haze                              198
Thunder in the Vicinity           195
Light Drizzle                     179
Light Snow / Windy                174
Smoke                             169
Light Rain with Thunder           158
Wintry Mix                        152
Thunder                            95
T-Storm                            92
Overcast                           89
Rain                               86
Light Freezing Drizzle          

In [460]:
df_drop['Weather_Condition'] = df_drop['Weather_Condition'].apply(lambda x: re.sub(r'[^a-zA-Z\s]','',str(x)))
df_drop['Weather_Condition'] = df_drop['Weather_Condition'].str.replace(" ", "")

pd.options.display.max_rows=100
df_drop['Weather_Condition'].value_counts()

/var/folders/s7/d5dl_ywn63zg7wx1h9wckr2r0000gn/T/ipykernel_1285/344051574.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop['Weather_Condition'] = df_drop['Weather_Condition'].apply(lambda x: re.sub(r'[^a-zA-Z\s]','',str(x)))
/var/folders/s7/d5dl_ywn63zg7wx1h9wckr2r0000gn/T/ipykernel_1285/344051574.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop['Weather_Condition'] = df_drop['Weather_Condition'].str.replace(" ", "")


Weather_Condition
Fair                         23317
MostlyCloudy                  9950
PartlyCloudy                  8396
LightSnow                     5561
Cloudy                        5353
FairWindy                     1017
Snow                           801
LightRain                      760
MostlyCloudyWindy              551
PartlyCloudyWindy              471
Fog                            440
ShowersintheVicinity           205
HeavySnow                      204
CloudyWindy                    200
Haze                           198
ThunderintheVicinity           195
LightDrizzle                   179
LightSnowWindy                 174
Smoke                          169
LightRainwithThunder           158
WintryMix                      152
Thunder                         95
TStorm                          92
Overcast                        89
Rain                            86
LightFreezingDrizzle            75
SnowWindy                       62
LightRainWindy                  48
NA

In [461]:
# df_drop['Weather_Condition'] = df_drop['Weather_Condition'].apply(lambda x: re.sub(r'[^a-zA-Z\s]','',str(x)))
# df_drop['Weather_Condition'].str.replace(" ", "")


df_drop['Weather_Condition'] = df_drop['Weather_Condition'].replace({'Overcast': 'Cloudy', 
                                                                'LightFreezingDrizzle': 'LightDrizzle', 
                                                                'SnowWindy':'LightSnowWindy',
                                                                'LightRainWindy': 'LightSnowWindy',
                                                                'NAPrecipitation': 'Rain',
                                                                'HeavySnowWindy': 'Snow',
                                                                'HazeWindy': 'Haze',
                                                                'LightRainShower': 'Rain',
                                                                'FogWindy': 'Fog',
                                                                'TStormWindy': 'TStorm',
                                                                'PatchesofFog': 'Fog',
                                                                'HeavyRain': 'Rain',
                                                                'ThunderWindy': 'Thunder',
                                                                'BlowingDustWindy': 'CloudyWindy',
                                                                'BlowingSnowWindy': 'MostlyCloudyWindy',
                                                                'HeavyTStorm': 'TStorm',
                                                                'LightFreezingFog': 'Fog',
                                                                'HeavyTStormWindy': 'TStorm',
                                                                'PartialFog': 'Fog',
                                                                'WidespreadDustWindy': 'Fog',
                                                                'ScatteredClouds': 'Cloudy',
                                                                'DrizzleandFog': 'Fog',
                                                                'RainWindy': 'Windy',
                                                                'SquallsWindy': 'Windy',
                                                                'BlowingSnow': 'Snow',
                                                                'SmokeWindy': 'Smoke',
                                                                'LightSnowwithThunder': 'LightSnow',
                                                                'HeavyRainWindy': 'Rain',
                                                                'LightFreezingRain': 'Rain',
                                                                'Clear': 'Fair',
                                                                'BlowingDust': 'MostlyCloudyWindy',
                                                                'LightRainShowerWindy': 'LightSnowWindy',
                                                                'HeavyDrizzle': 'LightDrizzle',
                                                                'ThunderWintryMix': 'WintryMix',
                                                                'LightDrizzleWindy': 'LightDrizzle',
                                                                'LightSnowShower': 'LightSnow',
                                                                'LightThunderstormsandSnow': 'LightSnow',
                                                                'LightThunderstormsandRain': 'LightRain',
                                                                'FreezingDrizzle': 'Drizzle',
                                                                'Squalls': 'FairWindy',
                                                                'Windy': 'FairWindy',
                                                                'Thunderstorm': 'Thunder',
                                                                'ColoudyWind': 'MostlyCloudyWindy',
                                                                'Drizzle': 'LightDrizzle',
                                                                'CloudyWindy': 'MostlyCloudyWindy',
                                                                'Clouds': 'Cloudy',
                                                                'LightSnowWindy': 'FairWindy',
                                                                'LightSnowWindy': 'LightSnowWindy'
                                                    })

pd.options.display.max_rows=100
df_drop['Weather_Condition'].value_counts()


/var/folders/s7/d5dl_ywn63zg7wx1h9wckr2r0000gn/T/ipykernel_1285/2184661440.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop['Weather_Condition'] = df_drop['Weather_Condition'].replace({'Overcast': 'Cloudy',


Weather_Condition
Fair                    23321
MostlyCloudy             9950
PartlyCloudy             8396
LightSnow                5568
Cloudy                   5449
FairWindy                1018
Snow                      841
MostlyCloudyWindy         767
LightRain                 761
Fog                       513
PartlyCloudyWindy         471
LightSnowWindy            286
LightDrizzle              264
Haze                      231
ShowersintheVicinity      205
HeavySnow                 204
ThunderintheVicinity      195
Rain                      175
Smoke                     174
LightRainwithThunder      158
WintryMix                 154
TStorm                    134
Thunder                   112
CloudyWindy                16
Windy                      12
Drizzle                     1
Name: count, dtype: int64

In [462]:
# Preprocess Weather_Condition

# Preprocess "Weather_Condition" column (one-hot encoding)
weather_encoder = OneHotEncoder(sparse_output=False)
weather_encoded = weather_encoder.fit_transform(df[['Weather_Condition']])
weather_columns = weather_encoder.get_feature_names_out(['Weather_Condition'])
df_weather_encoded = pd.DataFrame(weather_encoded, columns=weather_columns)
df_weather_encoded.head()


,Weather_Condition_Blowing Dust,Weather_Condition_Blowing Dust / Windy,Weather_Condition_Blowing Snow,Weather_Condition_Blowing Snow / Windy,Weather_Condition_Clear,Weather_Condition_Cloudy,Weather_Condition_Cloudy / Windy,Weather_Condition_Drizzle,Weather_Condition_Drizzle and Fog,Weather_Condition_Fair,...,Weather_Condition_T-Storm / Windy,Weather_Condition_Thunder,Weather_Condition_Thunder / Windy,Weather_Condition_Thunder / Wintry Mix,Weather_Condition_Thunder in the Vicinity,Weather_Condition_Thunderstorm,Weather_Condition_Thunderstorms and Rain,Weather_Condition_Widespread Dust / Windy,Weather_Condition_Wintry Mix,Weather_Condition_nan
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [463]:
new_list = []

for column in df_drop.columns:
    if df_drop[column].dtypes == "bool":
        new_list.append(column)
new_list

['Amenity',
 'Bump',
 'Crossing',
 'Give_Way',
 'Junction',
 'No_Exit',
 'Railway',
 'Roundabout',
 'Station',
 'Stop',
 'Traffic_Calming',
 'Traffic_Signal',
 'Turning_Loop']

In [464]:
# Encoding the Bump column using a custom function

def encode_Bool(column):
    """
    This function encodes by setting yes as 1 and no as 0.
    """
    if column == "True":
        return 1
    else:
        return 0

# Call the encode_Bump function 
for column in new_list:
    df_drop[column] = df_drop[column].apply(encode_Bool)

# Review the DataFrame 

df_drop

/var/folders/s7/d5dl_ywn63zg7wx1h9wckr2r0000gn/T/ipykernel_1285/2427824266.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop[column] = df_drop[column].apply(encode_Bool)


,Unnamed: 0,Severity,Start_Time,End_Time,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,512415,3,-9223372036854775808,-9223372036854775808,93,93,11,25,10,NE,...,0,0,0,0,0,0,Day,Day,Day,Day
1,512525,1,-9223372036854775808,-9223372036854775808,86,86,13,25,10,NW,...,0,0,0,0,0,0,Day,Day,Day,Day
2,512588,2,-9223372036854775808,-9223372036854775808,71,71,26,25,10,SW,...,0,0,0,0,0,0,Day,Day,Day,Day
3,512593,1,-9223372036854775808,-9223372036854775808,78,78,21,25,10,SW,...,0,0,0,0,0,0,Day,Day,Day,Day
4,512594,3,-9223372036854775808,-9223372036854775808,86,86,13,25,10,NW,...,0,0,0,0,0,0,Day,Day,Day,Day
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90880,7728172,2,-9223372036854775808,-9223372036854775808,86,86,14,25,10,EAST,...,0,0,0,0,0,0,Day,Day,Day,Day
90881,7728181,3,-9223372036854775808,-9223372036854775808,80,80,27,25,10,SOUTH,...,0,0,0,0,0,0,Day,Day,Day,Day
90882,7728187,3,-9223372036854775808,-9223372036854775808,82,82,20,25,10,SE,...,0,0,0,0,0,0,Day,Day,Day,Day
90883,7728192,4,-9223372036854775808,-9223372036854775808,76,76,36,26,10,NE,...,0,0,0,0,0,0,Night,Night,Day,Day


In [465]:
Day_list = []
    
for column in df_drop.columns:
    if df_drop[column].dtypes == "object":
        Day_list.append(column)
Day_list

['Wind_Direction',
 'Weather_Condition',
 'Sunrise_Sunset',
 'Civil_Twilight',
 'Nautical_Twilight',
 'Astronomical_Twilight']

In [466]:
# Encoding the Bump column using a custom function

def encode_Day(column):
    """
    This function encodes by setting yes as 1 and no as 0.
    """
    if column == "Day":
        return 1
    else:
        return 0

# Call the encode_Bump function 
for column in Day_list:
    df_drop[column] = df_drop[column].apply(encode_Day)

# Review the DataFrame 

df_drop

/var/folders/s7/d5dl_ywn63zg7wx1h9wckr2r0000gn/T/ipykernel_1285/678044661.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_drop[column] = df_drop[column].apply(encode_Day)


,Unnamed: 0,Severity,Start_Time,End_Time,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,512415,3,-9223372036854775808,-9223372036854775808,93,93,11,25,10,0,...,0,0,0,0,0,0,1,1,1,1
1,512525,1,-9223372036854775808,-9223372036854775808,86,86,13,25,10,0,...,0,0,0,0,0,0,1,1,1,1
2,512588,2,-9223372036854775808,-9223372036854775808,71,71,26,25,10,0,...,0,0,0,0,0,0,1,1,1,1
3,512593,1,-9223372036854775808,-9223372036854775808,78,78,21,25,10,0,...,0,0,0,0,0,0,1,1,1,1
4,512594,3,-9223372036854775808,-9223372036854775808,86,86,13,25,10,0,...,0,0,0,0,0,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90880,7728172,2,-9223372036854775808,-9223372036854775808,86,86,14,25,10,0,...,0,0,0,0,0,0,1,1,1,1
90881,7728181,3,-9223372036854775808,-9223372036854775808,80,80,27,25,10,0,...,0,0,0,0,0,0,1,1,1,1
90882,7728187,3,-9223372036854775808,-9223372036854775808,82,82,20,25,10,0,...,0,0,0,0,0,0,1,1,1,1
90883,7728192,4,-9223372036854775808,-9223372036854775808,76,76,36,26,10,0,...,0,0,0,0,0,0,0,0,1,1


In [467]:
df_drop = pd.concat([ 
    df_drop,
    df_wind_encoded,
    df_weather_encoded
    ], axis=1)

df_drop = df_drop.drop(columns = ['Wind_Direction', 'Weather_Condition'])
df_drop.fillna(0, inplace=True)
df_drop

,Unnamed: 0,Severity,Start_Time,End_Time,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),...,Weather_Condition_T-Storm / Windy,Weather_Condition_Thunder,Weather_Condition_Thunder / Windy,Weather_Condition_Thunder / Wintry Mix,Weather_Condition_Thunder in the Vicinity,Weather_Condition_Thunderstorm,Weather_Condition_Thunderstorms and Rain,Weather_Condition_Widespread Dust / Windy,Weather_Condition_Wintry Mix,Weather_Condition_nan
0,512415.0,3.0,-9.223372e+18,-9.223372e+18,93.0,93.0,11.0,25.0,10.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,512525.0,1.0,-9.223372e+18,-9.223372e+18,86.0,86.0,13.0,25.0,10.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,512588.0,2.0,-9.223372e+18,-9.223372e+18,71.0,71.0,26.0,25.0,10.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,512593.0,1.0,-9.223372e+18,-9.223372e+18,78.0,78.0,21.0,25.0,10.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,512594.0,3.0,-9.223372e+18,-9.223372e+18,86.0,86.0,13.0,25.0,10.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90832,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90833,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90840,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90857,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [468]:
X = df_drop.drop(columns=['Severity'])
X.head()


,Unnamed: 0,Start_Time,End_Time,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),...,Weather_Condition_T-Storm / Windy,Weather_Condition_Thunder,Weather_Condition_Thunder / Windy,Weather_Condition_Thunder / Wintry Mix,Weather_Condition_Thunder in the Vicinity,Weather_Condition_Thunderstorm,Weather_Condition_Thunderstorms and Rain,Weather_Condition_Widespread Dust / Windy,Weather_Condition_Wintry Mix,Weather_Condition_nan
0,512415.0,-9.223372e+18,-9.223372e+18,93.0,93.0,11.0,25.0,10.0,8.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,512525.0,-9.223372e+18,-9.223372e+18,86.0,86.0,13.0,25.0,10.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,512588.0,-9.223372e+18,-9.223372e+18,71.0,71.0,26.0,25.0,10.0,13.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,512593.0,-9.223372e+18,-9.223372e+18,78.0,78.0,21.0,25.0,10.0,12.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,512594.0,-9.223372e+18,-9.223372e+18,86.0,86.0,13.0,25.0,10.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [469]:
y = df_drop['Severity']
y_counts = y_df.value_counts().sort_values(ascending=False)
y_counts


Severity
2.0    37705
0.0    31509
3.0    16123
4.0     4630
1.0      918
Name: count, dtype: int64

In [470]:
df_drop_corr = df_drop.corr()
df_drop_corr

,Unnamed: 0,Severity,Start_Time,End_Time,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),...,Weather_Condition_T-Storm / Windy,Weather_Condition_Thunder,Weather_Condition_Thunder / Windy,Weather_Condition_Thunder / Wintry Mix,Weather_Condition_Thunder in the Vicinity,Weather_Condition_Thunderstorm,Weather_Condition_Thunderstorms and Rain,Weather_Condition_Widespread Dust / Windy,Weather_Condition_Wintry Mix,Weather_Condition_nan
Unnamed: 0,1.000000,0.669386,-0.708671,-0.708671,0.543968,0.491333,0.519073,0.701359,0.448993,0.470796,...,0.010676,0.021876,0.010454,0.006215,0.026829,-0.018613,-0.009127,0.005520,0.014173,-0.122121
Severity,0.669386,1.000000,-0.908056,-0.908056,0.726676,0.664997,0.665044,0.904262,0.592525,0.572464,...,0.011376,0.023746,0.006499,0.005285,0.028186,-0.024987,-0.011696,0.004076,0.024385,-0.156480
Start_Time,-0.708671,-0.908056,1.000000,1.000000,-0.774545,-0.704115,-0.749843,-0.995319,-0.635147,-0.620120,...,-0.011335,-0.021252,-0.008273,-0.003417,-0.025216,0.027326,0.012880,-0.006835,-0.029816,0.172324
End_Time,-0.708671,-0.908056,1.000000,1.000000,-0.774545,-0.704115,-0.749843,-0.995319,-0.635147,-0.620120,...,-0.011335,-0.021252,-0.008273,-0.003417,-0.025216,0.027326,0.012880,-0.006835,-0.029816,0.172324
Temperature(F),0.543968,0.726676,-0.774545,-0.774545,1.000000,0.990789,0.295482,0.779027,0.620178,0.528209,...,0.021105,0.044248,0.017612,0.002051,0.056166,-0.021468,-0.009976,0.014660,-0.003530,-0.133473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Weather_Condition_Thunderstorm,-0.018613,-0.024987,0.027326,0.027326,-0.021468,-0.019673,-0.020297,-0.026916,-0.017350,-0.016400,...,-0.000322,-0.000684,-0.000283,-0.000097,-0.001014,1.000000,-0.000194,-0.000194,-0.000848,-0.002601
Weather_Condition_Thunderstorms and Rain,-0.009127,-0.011696,0.012880,0.012880,-0.009976,-0.009069,-0.009658,-0.012819,-0.008181,-0.007987,...,-0.000146,-0.000310,-0.000128,-0.000044,-0.000459,-0.000194,1.000000,-0.000088,-0.000384,-0.001178
Weather_Condition_Widespread Dust / Windy,0.005520,0.004076,-0.006835,-0.006835,0.014660,0.015019,-0.007099,0.006235,-0.003128,0.035742,...,-0.000146,-0.000310,-0.000128,-0.000044,-0.000459,-0.000194,-0.000088,1.000000,-0.000384,-0.001178
Weather_Condition_Wintry Mix,0.014173,0.024385,-0.029816,-0.029816,-0.003530,-0.009225,0.073026,0.029134,-0.010746,0.011342,...,-0.000637,-0.001352,-0.000560,-0.000192,-0.002002,-0.000848,-0.000384,-0.000384,1.000000,-0.005138


In [471]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [472]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)
# Scale the training and testing data

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [473]:
#Create the model
input_layer = layers.Input(shape=(X.shape[1],), name='input_features')

shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

In [474]:
severity_output = layers.Dense(1, activation='sigmoid', name='severity_output')(shared_layer2)

In [475]:
# Create the model
model = Model(inputs=input_layer, outputs=[severity_output])

# Compile the model
model.compile(optimizer='adam',
            loss={'severity_output': 'categorical_crossentropy'},
            metrics={'severity_output': 'accuracy'})

# Display the model summary
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_features (InputLayer)     │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         7,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ severity_output (Dense)         │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,345 (36.50 KB)

 Trainable params: 9,345 (36.50 KB)

 Non-trainable params: 0 (0.00 B)

In [478]:
# Fit the model
model.fit(
    X,
    {'severity_output':y },
    epochs=10,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/10


/Users/avalee/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/losses/losses.py:22: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


2273/2273 ━━━━━━━━━━━━━━━━━━━━ 2s 490us/step - accuracy: 0.1785 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epoch 2/10


/Users/avalee/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/losses/losses.py:22: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


2273/2273 ━━━━━━━━━━━━━━━━━━━━ 1s 511us/step - accuracy: 0.1829 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epoch 3/10
2273/2273 ━━━━━━━━━━━━━━━━━━━━ 1s 603us/step - accuracy: 0.1820 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epoch 4/10
2273/2273 ━━━━━━━━━━━━━━━━━━━━ 1s 459us/step - accuracy: 0.1826 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epoch 5/10
2273/2273 ━━━━━━━━━━━━━━━━━━━━ 1s 441us/step - accuracy: 0.1849 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epoch 6/10
2273/2273 ━━━━━━━━━━━━━━━━━━━━ 1s 449us/step - accuracy: 0.1837 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epoch 7/10
2273/2273 ━━━━━━━━━━━━━━━━━━━━ 1s 500us/step - accuracy: 0.1818 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epoch 8/10
2273/2273 ━━━━━━━━━━━━━━━━━━━━ 1s 501us/step - accuracy: 0.1832 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epoch 9/10
2273/2273 ━━━━━━━━━━━━━━━━━━━━ 1s 435us/step - accuracy: 0.1845 - loss: nan - val_accuracy: 1.0000 - val_loss: nan
Epo

In [480]:
# Evaluate the model with the testing data
test_results = model.evaluate(X_test, {'severity_output': y_test})
test_results

711/711 ━━━━━━━━━━━━━━━━━━━━ 0s 276us/step - accuracy: 0.3458 - loss: nan


[nan, 0.34442391991615295]

In [485]:
# Print the quality and color accuracy
print(f"Severity Accuracy: {test_results[1]}")

Severity Accuracy: 0.34442391991615295


In [476]:
# non_numeric = df_drop['Severity'].apply(lambda x: isinstance(x, str))
# print(df_drop[non_numeric])